In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
from tqdm import tqdm

In [3]:
files = os.listdir('./data/kospidata/')
print(files)

['20210628~20210705.csv', '20210706~20210712.csv', '20210713~20210719.csv', '20210720~20210726.csv', '20210727~20210802.csv', '20210803~20210809.csv', '20210810~20210816.csv', '20210817~20210823.csv', '20210824~20210830.csv', '20210831~20210906.csv', '20210907~20210913.csv', '20210914~20210921.csv', '20210922~20210928.csv', '20210929~20211004.csv', '20211005~20211011.csv', '20211012~20211018.csv', '20211019~20211025.csv', '20211026~20211101.csv', '20211102~20211108.csv', '20211109~20211115.csv', '20211116~20211122.csv', '20211123~20211129.csv', '20211130~20211206.csv', '20211207~20211213.csv', '20211214~20211220.csv', '20211221~20211227.csv', '20211228~20220103.csv', '20220104~20220110.csv', '20220111~20220117.csv', '20220118~20220124.csv', '20220125~20220131.csv', '20220201~20220207.csv', '20220208~20220214.csv', '20220215~20220221.csv', '20220222~20220301.csv', '20220302~20220307.csv', '20220308~20220314.csv', '20220315~20220321.csv', '20220322~20220328.csv', '20220329~20220404.csv',

In [4]:
files[0][:-4]

'20210628~20210705'

In [5]:
df = pd.read_csv('./data/kospidata/20210628~20210705.csv',header=None)
df.columns = ['날짜','시간','고가','저가','종가']
df

,날짜,시간,고가,저가,종가
0,20210705,1530,3293.729980,3293.080078,3293.209961
1,20210705,1529,3293.729980,3293.729980,3293.729980
2,20210705,1528,3293.729980,3293.729980,3293.729980
3,20210705,1527,3293.729980,3293.729980,3293.729980
4,20210705,1526,3293.729980,3293.729980,3293.729980
...,...,...,...,...,...
2335,20210628,905,3299.639893,3298.590088,3298.739990
2336,20210628,904,3302.820068,3300.129883,3300.129883
2337,20210628,903,3304.020020,3302.399902,3302.909912
2338,20210628,902,3307.860107,3302.389893,3302.969971


In [6]:
df_all = pd.DataFrame()
i=0
for file in files[:-1]:
    
    globals()['df_{}_'.format(i)] = pd.read_csv('./data/kospidata/{}'.format(file), header=None)
    globals()['df_{}_'.format(i)].columns = ['날짜','시간','고가','저가','종가']
    df_all = pd.concat([df_all, globals()['df_{}_'.format(i)]])
    df_all.reset_index(inplace=True,drop=True)
    i+=1

In [7]:
df_all['날짜'] = pd.to_datetime(df_all['날짜'],format='%Y%m%d')

In [8]:
df_all.sort_values(by='날짜',inplace=True)

In [9]:
dates = df_all['날짜'].dt.date.unique()

In [ ]:
dates

In [11]:
testdf = pd.DataFrame()

for date in dates:
    daily_data = df_all[df_all['날짜'].dt.date == date].copy()

    daily_data['종가'] = daily_data['종가'].shift(1)
    daily_data = daily_data.dropna()

    testdf = pd.concat([testdf,daily_data])

In [12]:
testdf.reset_index(drop=True,inplace=True)

In [13]:
# 9시00분~15시20분 외 데이터 자르기
i=0
for time_ in testdf['시간'].astype('str') :
    time_ = datetime.strptime(time_, '%H%M').time()
    if (time_ > datetime.strptime('1520','%H%M').time()) or (time_ < datetime.strptime('0900','%H%M').time()) :
        testdf.drop(index=[i],axis=1,inplace=True)
        i+=1
    else:
        i+=1

In [14]:
len(testdf['시간'].unique())

380

In [16]:
# 날짜 + 시간 = Datetime 컬럼  추가
testdf.insert(0,'Datetime',testdf['날짜'].astype('str') + testdf['시간'].astype('str'))
testdf['Datetime'] = testdf['Datetime'].apply(lambda _ : datetime.strptime(_,'%Y-%m-%d%H%M'))

# 시간순 정렬
testdf.sort_values('Datetime',inplace=True,ignore_index=True)


In [17]:
testdf

,Datetime,날짜,시간,고가,저가,종가
0,2021-06-28 09:01:00,2021-06-28,901,3308.520020,3304.860107,3302.969971
1,2021-06-28 09:02:00,2021-06-28,902,3307.860107,3302.389893,3302.909912
2,2021-06-28 09:03:00,2021-06-28,903,3304.020020,3302.399902,3300.129883
3,2021-06-28 09:04:00,2021-06-28,904,3302.820068,3300.129883,3298.739990
4,2021-06-28 09:05:00,2021-06-28,905,3299.639893,3298.590088,3298.919922
...,...,...,...,...,...,...
193478,2023-07-20 15:16:00,2023-07-20,1516,2605.949951,2604.090088,2605.719971
193479,2023-07-20 15:17:00,2023-07-20,1517,2604.280029,2603.479980,2604.090088
193480,2023-07-20 15:18:00,2023-07-20,1518,2604.010010,2602.989990,2603.939941
193481,2023-07-20 15:19:00,2023-07-20,1519,2604.479980,2603.570068,2603.899902


In [19]:
ex_df = pd.read_csv('./data/환율데이터/exchange_rate_CNY.csv')

In [20]:
testdf['Datetime'][0]

Timestamp('2021-06-28 09:01:00')

In [21]:
ex_df['Datetime'] = ex_df['Datetime'].apply(lambda _ : datetime.strptime(_,'%Y-%m-%d %H:%M:%S'))

In [22]:
ex_df['Datetime'][0]

Timestamp('2021-06-28 09:01:00')

In [23]:
abtimes = list(set(ex_df['Datetime']) - set(testdf['Datetime']))

In [24]:
len(abtimes)

697

In [25]:
testdf_ = pd.DataFrame(columns=testdf.columns)
testdf_['Datetime'] =  abtimes

df_kospi = pd.concat([testdf,testdf_])

In [26]:
df_kospi

,Datetime,날짜,시간,고가,저가,종가
0,2021-06-28 09:01:00,2021-06-28,901,3308.520020,3304.860107,3302.969971
1,2021-06-28 09:02:00,2021-06-28,902,3307.860107,3302.389893,3302.909912
2,2021-06-28 09:03:00,2021-06-28,903,3304.020020,3302.399902,3300.129883
3,2021-06-28 09:04:00,2021-06-28,904,3302.820068,3300.129883,3298.739990
4,2021-06-28 09:05:00,2021-06-28,905,3299.639893,3298.590088,3298.919922
...,...,...,...,...,...,...
692,2023-04-10 13:20:00,NaT,NaN,NaN,NaN,NaN
693,2023-01-31 13:20:00,NaT,NaN,NaN,NaN,NaN
694,2023-03-27 09:01:00,NaT,NaN,NaN,NaN,NaN
695,2023-01-02 09:06:00,NaT,NaN,NaN,NaN,NaN


In [27]:
df_kospi.sort_values(by='Datetime', inplace=True)
df_kospi = df_kospi.reindex()

df_kospi['날짜'].ffill(inplace=True)

df_kospi['고가'].bfill(inplace=True)
df_kospi['저가'].bfill(inplace=True)
df_kospi['종가'].bfill(inplace=True)

In [28]:
df_kospi = df_kospi[['Datetime','고가','저가','종가']]

In [29]:
df_kospi.to_csv('./data/kospi.csv',index=False)

In [30]:
df_kospi

,Datetime,고가,저가,종가
0,2021-06-28 09:01:00,3308.520020,3304.860107,3302.969971
1,2021-06-28 09:02:00,3307.860107,3302.389893,3302.909912
2,2021-06-28 09:03:00,3304.020020,3302.399902,3300.129883
3,2021-06-28 09:04:00,3302.820068,3300.129883,3298.739990
4,2021-06-28 09:05:00,3299.639893,3298.590088,3298.919922
...,...,...,...,...
193478,2023-07-20 15:16:00,2605.949951,2604.090088,2605.719971
193479,2023-07-20 15:17:00,2604.280029,2603.479980,2604.090088
193480,2023-07-20 15:18:00,2604.010010,2602.989990,2603.939941
193481,2023-07-20 15:19:00,2604.479980,2603.570068,2603.899902
